In [47]:
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import timedelta
import datetime as dt
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 10]
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
import warnings
from pandas.tseries.holiday import USFederalHolidayCalendar
#from geopy import distance 
#import haversine
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

In [48]:
train.head()

,direction,distance_dummy_manhattan,distance_haversine,dropoff_cluster,dropoff_datetime,dropoff_latitude,dropoff_longitude,dropoff_pca0,dropoff_pca1,holiday,holiday_after,holiday_prev,id,log_trip_duration,log_trip_duration_gby_dropoff_cluster,log_trip_duration_gby_pickup_cluster,log_trip_duration_gby_pickup_date,log_trip_duration_gby_pickup_hour,log_trip_duration_gby_pickup_week_hour,log_trip_duration_gby_vendor_id,log_trip_duration_gbys_pickup_cluster_dropoff_cluster,log_trip_duration_gbys_pickup_hour_dropoff_cluster,log_trip_duration_gbys_pickup_hour_pickup_cluster,passenger_count,pickup_cluster,pickup_date,pickup_datetime,pickup_hour,pickup_hour_weekofyear,pickup_latitude,pickup_longitude,pickup_minute,pickup_pca0,pickup_pca1,pickup_week_hour,pickup_weekday,speed,speed_gby_dropoff_cluster,speed_gby_pickup_cluster,speed_gby_pickup_date,speed_gby_pickup_hour,speed_gby_pickup_week_hour,speed_gby_vendor_id,speed_gbys_pickup_cluster_dropoff_cluster,speed_gbys_pickup_hour_dropoff_cluster,speed_gbys_pickup_hour_pickup_cluster,store_and_fwd_flag,trip_duration,vendor_id,total_trips_gby_pickup_cluster,total_trips_gby_dropoff_cluster,total_trips_gby_pickup_hour,total_trips_gby_pickup_week_hour,total_trips_gby_pickup_date,tempi,wspdi,visi,rain,snow,total_distance,total_travel_time,number_of_steps
0,99.970196,1.735433,1.498521,18,2016-03-14 17:32:30,40.765602,-73.964630,-0.009995,0.013490,0,0,0,id2875421,6.122493,6.326557,6.432302,6.443767,6.533998,6.470036,6.474120,6.413152,6.434881,6.521501,1,73,2016-03-14,2016-03-14 17:24:55,17,11,40.767937,-73.982155,24,0.007345,0.016941,17,0,0.003814,0.004642,0.004474,0.004976,0.004488,0.004664,0.005188,0.003407,0.003977,0.003891,0,455.0,2,31266,40911,108274,15525,11465,41.0,13.8,8.0,0.0,0.0,2009.1,164.9,5
1,-117.153768,2.430506,1.805507,15,2016-06-12 00:54:38,40.731152,-73.999481,0.026989,-0.018660,0,0,0,id2377394,6.498282,6.343346,6.368840,6.437409,6.435587,6.475604,6.467068,6.651564,6.264913,6.289805,1,51,2016-06-12,2016-06-12 00:43:35,0,23,40.738564,-73.980415,43,0.007488,-0.012483,144,6,0.003666,0.004673,0.005179,0.006028,0.006327,0.005590,0.005183,0.004198,0.004877,0.006357,0,663.0,1,27944,41858,75524,17275,10023,41.0,3.5,10.0,0.0,0.0,2513.2,332.0,6
2,-159.680165,8.203575,6.385098,6,2016-01-19 12:10:48,40.710087,-74.005333,0.034177,-0.039307,0,1,0,id3858529,7.661527,6.715509,6.559381,6.462319,6.527820,6.645506,6.474120,7.326867,6.850751,6.628287,1,3,2016-01-19,2016-01-19 11:35:24,11,3,40.763939,-73.979027,35,0.004479,0.012751,35,1,0.003862,0.005745,0.004312,0.004984,0.004473,0.003856,0.005188,0.005812,0.005090,0.003877,0,2124.0,2,48977,27043,97047,13847,11244,24.1,16.1,10.0,0.0,0.0,11060.8,767.6,16
3,-172.737700,1.661331,1.485498,61,2016-04-06 19:39:40,40.706718,-74.012268,0.041313,-0.042225,0,0,0,id3504673,6.063785,6.734042,6.525633,6.499305,6.448123,6.465914,6.474120,6.327196,6.643812,6.490176,1,49,2016-04-06,2016-04-06 19:32:31,19,14,40.719971,-74.010040,32,0.038242,-0.029143,67,2,0.003873,0.006476,0.005099,0.004867,0.004881,0.004703,0.005188,0.004571,0.006099,0.004806,0,429.0,2,31233,22844,128184,19446,11964,72.0,3.5,1.8,1.0,0.0,1779.4,235.8,4
4,179.473585,1.199457,1.188588,56,2016-03-26 13:38:10,40.782520,-73.972923,-0.002801,0.030904,0,0,0,id2181028,6.077642,6.227874,6.297403,6.404304,6.539142,6.442932,6.474120,5.444955,6.206583,6.296941,1,10,2016-03-26,2016-03-26 13:30:55,13,12,40.793209,-73.973053,30,-0.003355,0.041579,133,5,0.002757,0.005484,0.005555,0.005582,0.004477,0.004851,0.005188,0.004221,0.005005,0.005150,0,435.0,2,19848,32902,101308,16220,12062,54.0,6.9,10.0,0.0,0.0,1614.9,140.1,5


In [49]:
print('Loading data')
weather = pd.read_csv('input/new-york-city-taxi-trip-hourly-weather-data/Weather.csv',
                      parse_dates=['pickup_datetime'])

osrm_test = pd.read_csv('input/new-york-city-taxi-with-osrm/fastest_routes_test.csv')
orsm_train1 = pd.read_csv('input/new-york-city-taxi-with-osrm/fastest_routes_train_part_1.csv')
orsm_train2 = pd.read_csv('input/new-york-city-taxi-with-osrm/fastest_routes_train_part_2.csv')

test = pd.read_csv('input/nyc-taxi-trip-duration/test.csv',
                   parse_dates=['pickup_datetime'])

train = pd.read_csv('input/nyc-taxi-trip-duration/train.csv',
                   parse_dates=['pickup_datetime', 'dropoff_datetime'])

Loading data


In [50]:
weather = weather[['pickup_datetime', 'tempi', 'wspdi', 'visi', 'rain', 'snow']]
weather.loc[:, 'pickup_date'] = weather['pickup_datetime'].dt.date
weather.loc[:, 'pickup_hour'] = weather['pickup_datetime'].dt.hour
# get hourly data and fill nan with nearest non-NaN data
weather_hourly = weather.groupby(['pickup_date', 'pickup_hour'])['tempi', 'wspdi', 'visi', 'rain', 'snow']\
                        .mean().round(decimals=1).reset_index().interpolate(method='nearest')
weather_hourly.head()

,pickup_date,pickup_hour,tempi,wspdi,visi,rain,snow
0,2015-12-31,0,46.0,4.2,3.8,0.3,0.0
1,2015-12-31,1,45.0,4.6,8.0,0.0,0.0
2,2015-12-31,2,45.0,0.0,8.0,0.0,0.0
3,2015-12-31,3,44.7,3.1,8.3,0.0,0.0
4,2015-12-31,4,45.0,3.5,8.0,0.0,0.0


In [51]:
train.loc[:, 'log_trip_duration'] = np.log1p(train['trip_duration'].values)

In [52]:
train.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,log_trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,6.122493
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,6.498282
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,7.661527
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,6.063785
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,6.077642


In [53]:
print(train.trip_duration.quantile(0.995))
print(train.trip_duration.quantile(0.005))
train.trip_duration.describe()

4139.0
50.0


count    1.458644e+06
mean     9.594923e+02
std      5.237432e+03
min      1.000000e+00
25%      3.970000e+02
50%      6.620000e+02
75%      1.075000e+03
max      3.526282e+06
Name: trip_duration, dtype: float64

In [54]:
# min_trip_dur = train.log_trip_duration.quantile(0.005)
# max_trip_dur = train.log_trip_duration.quantile(0.995)
# train = train.loc[(train['log_trip_duration'] < max_trip_dur) & (train['log_trip_duration'] > min_trip_dur)]

In [55]:
# get all log/lat available in train/test
coords = np.vstack((train[['pickup_latitude', 'pickup_longitude']].values,
                    train[['dropoff_latitude', 'dropoff_longitude']].values,
                    test[['pickup_latitude', 'pickup_longitude']].values,
                    test[['dropoff_latitude', 'dropoff_longitude']].values))

In [56]:
# get a randome N inside len(coords), only fit cluster to the sample data 
sample_ind = np.random.permutation(len(coords))[:500000] 
kmeans = MiniBatchKMeans(n_clusters=100, batch_size=10000).fit(coords[sample_ind])

train.loc[:, 'pickup_cluster'] = kmeans.predict(train[['pickup_latitude', 'pickup_longitude']])
train.loc[:, 'dropoff_cluster'] = kmeans.predict(train[['dropoff_latitude', 'dropoff_longitude']])
test.loc[:, 'pickup_cluster'] = kmeans.predict(test[['pickup_latitude', 'pickup_longitude']])
test.loc[:, 'dropoff_cluster'] = kmeans.predict(test[['dropoff_latitude', 'dropoff_longitude']])

In [57]:
train.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,log_trip_duration,pickup_cluster,dropoff_cluster
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,6.122493,7,19
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,6.498282,10,28
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,7.661527,62,88
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,6.063785,5,30
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,6.077642,74,18


### PCA 

In [58]:
pca = PCA().fit(coords)
train['pickup_pca0'] = pca.transform(train[['pickup_latitude', 'pickup_longitude']])[:, 0]
train['pickup_pca1'] = pca.transform(train[['pickup_latitude', 'pickup_longitude']])[:, 1]
train['dropoff_pca0'] = pca.transform(train[['dropoff_latitude', 'dropoff_longitude']])[:, 0]
train['dropoff_pca1'] = pca.transform(train[['dropoff_latitude', 'dropoff_longitude']])[:, 1]
test['pickup_pca0'] = pca.transform(test[['pickup_latitude', 'pickup_longitude']])[:, 0]
test['pickup_pca1'] = pca.transform(test[['pickup_latitude', 'pickup_longitude']])[:, 1]
test['dropoff_pca0'] = pca.transform(test[['dropoff_latitude', 'dropoff_longitude']])[:, 0]
test['dropoff_pca1'] = pca.transform(test[['dropoff_latitude', 'dropoff_longitude']])[:, 1]

In [59]:
# https://www.kaggle.com/the1owl/easy-as-baking-a-cake-if-baking-a-cake-is-easy
cal = USFederalHolidayCalendar()
holidays = [d.date() for d in cal.holidays(start='2016-01-01', end='2016-06-30').to_pydatetime()]
#business = [d.date() for d in pandas.date_range('2016-01-01', '2016-06-30') if d not in pandas.bdate_range('2016-01-01', '2016-06-30')]
holidays_prev = [d + timedelta(days=-1) for d in holidays]
holidays_after = [d + timedelta(days=1) for d in holidays]

In [60]:
def get_datetime_feature(df):
    df.loc[:, 'pickup_weekday'] = df['pickup_datetime'].dt.weekday
    df.loc[:, 'pickup_hour_weekofyear'] = df['pickup_datetime'].dt.weekofyear
    df.loc[:, 'pickup_hour'] = df['pickup_datetime'].dt.hour
    df.loc[:, 'pickup_minute'] = df['pickup_datetime'].dt.minute
    df.loc[:, 'pickup_week_hour'] = df['pickup_weekday'] * 24 + df['pickup_hour']
    df.loc[:, 'pickup_date'] = df['pickup_datetime'].dt.date
    df.loc[:, 'holiday_after'] = df.pickup_date.map(lambda x: 1 if x in holidays_after else 0)
    df.loc[:, 'holiday_prev'] = df.pickup_date.map(lambda x: 1 if x in holidays_prev else 0)
    df.loc[:, 'holiday'] = df.pickup_date.map(lambda x: 1 if x in holidays else 0)

In [61]:
get_datetime_feature(train)
get_datetime_feature(test)
train.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,log_trip_duration,pickup_cluster,dropoff_cluster,pickup_pca0,pickup_pca1,dropoff_pca0,dropoff_pca1,pickup_weekday,pickup_hour_weekofyear,pickup_hour,pickup_minute,pickup_week_hour,pickup_date,holiday_after,holiday_prev,holiday
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,6.122493,7,19,0.007691,0.017053,-0.009666,0.013695,0,11,17,24,17,2016-03-14,0,0,0
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,6.498282,10,28,0.007677,-0.012371,0.027145,-0.018652,6,23,0,43,144,2016-06-12,0,0,0
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,7.661527,62,88,0.004803,0.012879,0.034222,-0.039337,1,3,11,35,35,2016-01-19,1,0,0
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,6.063785,5,30,0.038342,-0.029194,0.041343,-0.042293,2,14,19,32,67,2016-04-06,0,0,0
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,6.077642,74,18,-0.002877,0.041749,-0.002380,0.031071,5,12,13,30,133,2016-03-26,0,0,0


In [62]:
# ref: https://www.kaggle.com/gaborfodor/from-eda-to-the-top-lb-0-367
def haversine_array(lat1, lng1, lat2, lng2):
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    AVG_EARTH_RADIUS = 6371  # in km
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * AVG_EARTH_RADIUS * np.arcsin(np.sqrt(d))
    return h

def dummy_manhattan_distance(lat1, lng1, lat2, lng2):
    a = haversine_array(lat1, lng1, lat1, lng2)
    b = haversine_array(lat1, lng1, lat2, lng1)
    return a + b

def bearing_array(lat1, lng1, lat2, lng2):
    AVG_EARTH_RADIUS = 6371  # in km
    lng_delta_rad = np.radians(lng2 - lng1)
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    y = np.sin(lng_delta_rad) * np.cos(lat2)
    x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(lng_delta_rad)
    return np.degrees(np.arctan2(y, x))  

train.loc[:, 'distance_haversine'] = haversine_array(train['pickup_latitude'].values, train['pickup_longitude'].values, train['dropoff_latitude'].values, train['dropoff_longitude'].values)
train.loc[:, 'distance_dummy_manhattan'] = dummy_manhattan_distance(train['pickup_latitude'].values, train['pickup_longitude'].values, train['dropoff_latitude'].values, train['dropoff_longitude'].values)
train.loc[:, 'direction'] = bearing_array(train['pickup_latitude'].values, train['pickup_longitude'].values, train['dropoff_latitude'].values, train['dropoff_longitude'].values)

test.loc[:, 'distance_haversine'] = haversine_array(test['pickup_latitude'].values, test['pickup_longitude'].values, test['dropoff_latitude'].values, test['dropoff_longitude'].values)
test.loc[:, 'distance_dummy_manhattan'] = dummy_manhattan_distance(test['pickup_latitude'].values, test['pickup_longitude'].values, test['dropoff_latitude'].values, test['dropoff_longitude'].values)
test.loc[:, 'direction'] = bearing_array(test['pickup_latitude'].values, test['pickup_longitude'].values, test['dropoff_latitude'].values, test['dropoff_longitude'].values)

In [63]:
train.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,log_trip_duration,pickup_cluster,dropoff_cluster,pickup_pca0,pickup_pca1,dropoff_pca0,dropoff_pca1,pickup_weekday,pickup_hour_weekofyear,pickup_hour,pickup_minute,pickup_week_hour,pickup_date,holiday_after,holiday_prev,holiday,distance_haversine,distance_dummy_manhattan,direction
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,6.122493,7,19,0.007691,0.017053,-0.009666,0.013695,0,11,17,24,17,2016-03-14,0,0,0,1.498521,1.735433,99.970196
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,6.498282,10,28,0.007677,-0.012371,0.027145,-0.018652,6,23,0,43,144,2016-06-12,0,0,0,1.805507,2.430506,-117.153768
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,7.661527,62,88,0.004803,0.012879,0.034222,-0.039337,1,3,11,35,35,2016-01-19,1,0,0,6.385098,8.203575,-159.680165
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,6.063785,5,30,0.038342,-0.029194,0.041343,-0.042293,2,14,19,32,67,2016-04-06,0,0,0,1.485498,1.661331,-172.737700
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,6.077642,74,18,-0.002877,0.041749,-0.002380,0.031071,5,12,13,30,133,2016-03-26,0,0,0,1.188588,1.199457,179.473585


In [64]:
train.loc[:, 'speed'] = train.distance_dummy_manhattan / train.trip_duration

train.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,log_trip_duration,pickup_cluster,dropoff_cluster,pickup_pca0,pickup_pca1,dropoff_pca0,dropoff_pca1,pickup_weekday,pickup_hour_weekofyear,pickup_hour,pickup_minute,pickup_week_hour,pickup_date,holiday_after,holiday_prev,holiday,distance_haversine,distance_dummy_manhattan,direction,speed
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,6.122493,7,19,0.007691,0.017053,-0.009666,0.013695,0,11,17,24,17,2016-03-14,0,0,0,1.498521,1.735433,99.970196,0.003814
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,6.498282,10,28,0.007677,-0.012371,0.027145,-0.018652,6,23,0,43,144,2016-06-12,0,0,0,1.805507,2.430506,-117.153768,0.003666
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,7.661527,62,88,0.004803,0.012879,0.034222,-0.039337,1,3,11,35,35,2016-01-19,1,0,0,6.385098,8.203575,-159.680165,0.003862
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,6.063785,5,30,0.038342,-0.029194,0.041343,-0.042293,2,14,19,32,67,2016-04-06,0,0,0,1.485498,1.661331,-172.737700,0.003873
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,6.077642,74,18,-0.002877,0.041749,-0.002380,0.031071,5,12,13,30,133,2016-03-26,0,0,0,1.188588,1.199457,179.473585,0.002757


For SPEED and DURATION information, we can only get it from train df, so we use some of the features(e.g. time, locations...) to get the mean of SPEED and DUR based on those feature and merge the mean() to the test df. For other information, we can get it from both train and test, so we want to concact the two and groupby on the ALL df, eg. number of trips from one location...

In [65]:
# speed and trip_duration related mean
for gby_col in ['vendor_id', 'pickup_cluster', 'dropoff_cluster', 'pickup_hour', \
                'pickup_week_hour', 'pickup_date']:
    gby = train.groupby(gby_col)['speed', 'log_trip_duration'].mean()
    gby.columns = ['%s_gby_%s' % (col, gby_col) for col in gby.columns]
    train = train.merge(gby, how='left', left_on=gby_col, right_index=True)
    test = test.merge(gby, how='left', left_on=gby_col, right_index=True)

In [66]:
for gby_cols in [['pickup_hour', 'pickup_cluster'],
                 ['pickup_hour', 'dropoff_cluster'],
                 ['pickup_cluster', 'dropoff_cluster']]:
    gbys = train.groupby(gby_cols)['speed', 'log_trip_duration'].mean()
    print(gbys.columns)
    gbys.columns = ['%s_gbys_%s' % (col, '_'.join(gby_cols)) for col in gbys.columns]
    gbys = gbys.reset_index()
    train = train.merge(gbys, how='left', on=gby_cols)
    test = test.merge(gbys, how='left', on=gby_cols)

Index(['speed', 'log_trip_duration'], dtype='object')
Index(['speed', 'log_trip_duration'], dtype='object')
Index(['speed', 'log_trip_duration'], dtype='object')


In [67]:
train.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,log_trip_duration,pickup_cluster,dropoff_cluster,pickup_pca0,pickup_pca1,dropoff_pca0,dropoff_pca1,pickup_weekday,pickup_hour_weekofyear,pickup_hour,pickup_minute,pickup_week_hour,pickup_date,holiday_after,holiday_prev,holiday,distance_haversine,distance_dummy_manhattan,direction,speed,speed_gby_vendor_id,log_trip_duration_gby_vendor_id,speed_gby_pickup_cluster,log_trip_duration_gby_pickup_cluster,speed_gby_dropoff_cluster,log_trip_duration_gby_dropoff_cluster,speed_gby_pickup_hour,log_trip_duration_gby_pickup_hour,speed_gby_pickup_week_hour,log_trip_duration_gby_pickup_week_hour,speed_gby_pickup_date,log_trip_duration_gby_pickup_date,speed_gbys_pickup_hour_pickup_cluster,log_trip_duration_gbys_pickup_hour_pickup_cluster,speed_gbys_pickup_hour_dropoff_cluster,log_trip_duration_gbys_pickup_hour_dropoff_cluster,speed_gbys_pickup_cluster_dropoff_cluster,log_trip_duration_gbys_pickup_cluster_dropoff_cluster
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,6.122493,7,19,0.007691,0.017053,-0.009666,0.013695,0,11,17,24,17,2016-03-14,0,0,0,1.498521,1.735433,99.970196,0.003814,0.005192,6.481609,0.004523,6.455874,0.004180,6.381392,0.004496,6.537261,0.004700,6.469051,0.004966,6.445390,0.003837,6.554691,0.003750,6.442969,0.003821,6.160872
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,6.498282,10,28,0.007677,-0.012371,0.027145,-0.018652,6,23,0,43,144,2016-06-12,0,0,0,1.805507,2.430506,-117.153768,0.003666,0.005167,6.450148,0.004952,6.362313,0.004712,6.288792,0.006315,6.422417,0.005577,6.465794,0.006094,6.421458,0.005878,6.305238,0.005073,6.177833,0.003321,6.333634
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,7.661527,62,88,0.004803,0.012879,0.034222,-0.039337,1,3,11,35,35,2016-01-19,1,0,0,6.385098,8.203575,-159.680165,0.003862,0.005192,6.481609,0.004360,6.534438,0.005179,6.544065,0.004470,6.520644,0.003850,6.635141,0.004975,6.459165,0.004030,6.549867,0.004512,6.664105,0.005667,7.315407
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,6.063785,5,30,0.038342,-0.029194,0.041343,-0.042293,2,14,19,32,67,2016-04-06,0,0,0,1.485498,1.661331,-172.737700,0.003873,0.005192,6.481609,0.005106,6.529342,0.006205,6.726676,0.004876,6.441269,0.004696,6.459194,0.004863,6.486666,0.004844,6.507031,0.005780,6.676450,0.003796,6.378557
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,6.077642,74,18,-0.002877,0.041749,-0.002380,0.031071,5,12,13,30,133,2016-03-26,0,0,0,1.188588,1.199457,179.473585,0.002757,0.005192,6.481609,0.005635,6.289475,0.005573,6.225785,0.004474,6.537807,0.004838,6.444176,0.005576,6.398234,0.005220,6.290428,0.005156,6.199833,0.004141,5.437386


In [68]:
df = pd.concat([train, test])
df.head(-5)

,direction,distance_dummy_manhattan,distance_haversine,dropoff_cluster,dropoff_datetime,dropoff_latitude,dropoff_longitude,dropoff_pca0,dropoff_pca1,holiday,holiday_after,holiday_prev,id,log_trip_duration,log_trip_duration_gby_dropoff_cluster,log_trip_duration_gby_pickup_cluster,log_trip_duration_gby_pickup_date,log_trip_duration_gby_pickup_hour,log_trip_duration_gby_pickup_week_hour,log_trip_duration_gby_vendor_id,log_trip_duration_gbys_pickup_cluster_dropoff_cluster,log_trip_duration_gbys_pickup_hour_dropoff_cluster,log_trip_duration_gbys_pickup_hour_pickup_cluster,passenger_count,pickup_cluster,pickup_date,pickup_datetime,pickup_hour,pickup_hour_weekofyear,pickup_latitude,pickup_longitude,pickup_minute,pickup_pca0,pickup_pca1,pickup_week_hour,pickup_weekday,speed,speed_gby_dropoff_cluster,speed_gby_pickup_cluster,speed_gby_pickup_date,speed_gby_pickup_hour,speed_gby_pickup_week_hour,speed_gby_vendor_id,speed_gbys_pickup_cluster_dropoff_cluster,speed_gbys_pickup_hour_dropoff_cluster,speed_gbys_pickup_hour_pickup_cluster,store_and_fwd_flag,trip_duration,vendor_id
0,99.970196,1.735433,1.498521,19,2016-03-14 17:32:30,40.765602,-73.964630,-0.009666,0.013695,0,0,0,id2875421,6.122493,6.381392,6.455874,6.445390,6.537261,6.469051,6.481609,6.160872,6.442969,6.554691,1,7,2016-03-14,2016-03-14 17:24:55,17,11,40.767937,-73.982155,24,0.007691,0.017053,17,0,0.003814,0.004180,0.004523,0.004966,0.004496,0.004700,0.005192,0.003821,0.003750,0.003837,N,455.0,2
1,-117.153768,2.430506,1.805507,28,2016-06-12 00:54:38,40.731152,-73.999481,0.027145,-0.018652,0,0,0,id2377394,6.498282,6.288792,6.362313,6.421458,6.422417,6.465794,6.450148,6.333634,6.177833,6.305238,1,10,2016-06-12,2016-06-12 00:43:35,0,23,40.738564,-73.980415,43,0.007677,-0.012371,144,6,0.003666,0.004712,0.004952,0.006094,0.006315,0.005577,0.005167,0.003321,0.005073,0.005878,N,663.0,1
2,-159.680165,8.203575,6.385098,88,2016-01-19 12:10:48,40.710087,-74.005333,0.034222,-0.039337,0,1,0,id3858529,7.661527,6.544065,6.534438,6.459165,6.520644,6.635141,6.481609,7.315407,6.664105,6.549867,1,62,2016-01-19,2016-01-19 11:35:24,11,3,40.763939,-73.979027,35,0.004803,0.012879,35,1,0.003862,0.005179,0.004360,0.004975,0.004470,0.003850,0.005192,0.005667,0.004512,0.004030,N,2124.0,2
3,-172.737700,1.661331,1.485498,30,2016-04-06 19:39:40,40.706718,-74.012268,0.041343,-0.042293,0,0,0,id3504673,6.063785,6.726676,6.529342,6.486666,6.441269,6.459194,6.481609,6.378557,6.676450,6.507031,1,5,2016-04-06,2016-04-06 19:32:31,19,14,40.719971,-74.010040,32,0.038342,-0.029194,67,2,0.003873,0.006205,0.005106,0.004863,0.004876,0.004696,0.005192,0.003796,0.005780,0.004844,N,429.0,2
4,179.473585,1.199457,1.188588,18,2016-03-26 13:38:10,40.782520,-73.972923,-0.002380,0.031071,0,0,0,id2181028,6.077642,6.225785,6.289475,6.398234,6.537807,6.444176,6.481609,5.437386,6.199833,6.290428,1,74,2016-03-26,2016-03-26 13:30:55,13,12,40.793209,-73.973053,30,-0.002877,0.041749,133,5,0.002757,0.005573,0.005635,0.005576,0.004474,0.004838,0.005192,0.004141,0.005156,0.005220,N,435.0,2
5,-44.995596,1.554180,1.098942,73,2016-01-30 22:09:03,40.749184,-73.992081,0.018700,-0.001085,0,0,0,id0801584,6.095825,6.459847,6.418260,6.425757,6.466885,6.495427,6.481609,6.195909,6.318115,6.443118,6,71,2016-01-30,2016-01-30 22:01:40,22,4,40.742195,-73.982857,1,0.009902,-0.008603,142,5,0.003508,0.004131,0.004616,0.005153,0.005719,0.005018,0.005192,0.003139,0.004508,0.005010,N,443.0,2
6,47.505775,1.873902,1.326279,65,2016-06-17 22:40:40,40.765896,-73.957405,-0.016896,0.013565,0,0,0,id1813257,5.834811,6.459215,6.348228,6.521516,6.466885,6.512968,6.450148,5.974696,6.409370,6.376672,4,81,2016-06-17,2016-06-17 22:34:59,22,24,40.757839,-73.969017,34,-0.004832,0.006203,118,4,0.005495,0.004744,0.005418,0.004865,0.005719,0.005183,0.005167,0.005181,0.005607,0.006010,N,341.0,1
7,136.385396,8.078684,5.714981,40,2016-05-21 08:20:49,40.760559,-73.922470,-0.051458,0.006188,0,0,0,id1324603,7.347300,6.821823,6.326963,6.497825,6.349027,6.169255,6.481609,7.384285,6.412848,

In [69]:
# count trips 
for gby_col in ['pickup_cluster', 'dropoff_cluster', 'pickup_hour', \
                'pickup_week_hour', 'pickup_date']:
    gby = df.groupby(gby_col)['id'].count().to_frame() # count the total trip based on time and location
    gby.columns = ['%s_gby_%s' % ('total_trips', gby_col) for col in gby.columns]
    df = df.merge(gby, how='left', left_on=gby_col, right_index=True)
    

In [70]:
weather.drop('pickup_datetime', axis=1, inplace=True)
weather.drop_duplicates(subset=['pickup_date', 'pickup_hour'], inplace=True)
df = df.merge(weather, on=['pickup_date', 'pickup_hour'], how='left')

In [71]:
#weather.head(30)

In [72]:
df.head()

,direction,distance_dummy_manhattan,distance_haversine,dropoff_cluster,dropoff_datetime,dropoff_latitude,dropoff_longitude,dropoff_pca0,dropoff_pca1,holiday,holiday_after,holiday_prev,id,log_trip_duration,log_trip_duration_gby_dropoff_cluster,log_trip_duration_gby_pickup_cluster,log_trip_duration_gby_pickup_date,log_trip_duration_gby_pickup_hour,log_trip_duration_gby_pickup_week_hour,log_trip_duration_gby_vendor_id,log_trip_duration_gbys_pickup_cluster_dropoff_cluster,log_trip_duration_gbys_pickup_hour_dropoff_cluster,log_trip_duration_gbys_pickup_hour_pickup_cluster,passenger_count,pickup_cluster,pickup_date,pickup_datetime,pickup_hour,pickup_hour_weekofyear,pickup_latitude,pickup_longitude,pickup_minute,pickup_pca0,pickup_pca1,pickup_week_hour,pickup_weekday,speed,speed_gby_dropoff_cluster,speed_gby_pickup_cluster,speed_gby_pickup_date,speed_gby_pickup_hour,speed_gby_pickup_week_hour,speed_gby_vendor_id,speed_gbys_pickup_cluster_dropoff_cluster,speed_gbys_pickup_hour_dropoff_cluster,speed_gbys_pickup_hour_pickup_cluster,store_and_fwd_flag,trip_duration,vendor_id,total_trips_gby_pickup_cluster,total_trips_gby_dropoff_cluster,total_trips_gby_pickup_hour,total_trips_gby_pickup_week_hour,total_trips_gby_pickup_date,tempi,wspdi,visi,rain,snow
0,99.970196,1.735433,1.498521,19,2016-03-14 17:32:30,40.765602,-73.964630,-0.009666,0.013695,0,0,0,id2875421,6.122493,6.381392,6.455874,6.445390,6.537261,6.469051,6.481609,6.160872,6.442969,6.554691,1,7,2016-03-14,2016-03-14 17:24:55,17,11,40.767937,-73.982155,24,0.007691,0.017053,17,0,0.003814,0.004180,0.004523,0.004966,0.004496,0.004700,0.005192,0.003821,0.003750,0.003837,N,455.0,2,56201,53443,109399,15664,11564,41.0,13.8,8.0,0.0,0.0
1,-117.153768,2.430506,1.805507,28,2016-06-12 00:54:38,40.731152,-73.999481,0.027145,-0.018652,0,0,0,id2377394,6.498282,6.288792,6.362313,6.421458,6.422417,6.465794,6.450148,6.333634,6.177833,6.305238,1,10,2016-06-12,2016-06-12 00:43:35,0,23,40.738564,-73.980415,43,0.007677,-0.012371,144,6,0.003666,0.004712,0.004952,0.006094,0.006315,0.005577,0.005167,0.003321,0.005073,0.005878,N,663.0,1,35903,25396,75962,17358,10086,41.0,3.5,10.0,0.0,0.0
2,-159.680165,8.203575,6.385098,88,2016-01-19 12:10:48,40.710087,-74.005333,0.034222,-0.039337,0,1,0,id3858529,7.661527,6.544065,6.534438,6.459165,6.520644,6.635141,6.481609,7.315407,6.664105,6.549867,1,62,2016-01-19,2016-01-19 11:35:24,11,3,40.763939,-73.979027,35,0.004803,0.012879,35,1,0.003862,0.005179,0.004360,0.004975,0.004470,0.003850,0.005192,0.005667,0.004512,0.004030,N,2124.0,2,56725,25421,97621,13937,11315,24.1,16.1,10.0,0.0,0.0
3,-172.737700,1.661331,1.485498,30,2016-04-06 19:39:40,40.706718,-74.012268,0.041343,-0.042293,0,0,0,id3504673,6.063785,6.726676,6.529342,6.486666,6.441269,6.459194,6.481609,6.378557,6.676450,6.507031,1,5,2016-04-06,2016-04-06 19:32:31,19,14,40.719971,-74.010040,32,0.038342,-0.029194,67,2,0.003873,0.006205,0.005106,0.004863,0.004876,0.004696,0.005192,0.003796,0.005780,0.004844,N,429.0,2,29748,35647,128745,19519,12045,72.0,3.5,1.8,1.0,0.0
4,179.473585,1.199457,1.188588,18,2016-03-26 13:38:10,40.782520,-73.972923,-0.002380,0.031071,0,0,0,id2181028,6.077642,6.225785,6.289475,6.398234,6.537807,6.444176,6.481609,5.437386,6.199833,6.290428,1,74,2016-03-26,2016-03-26 13:30:55,13,12,40.793209,-73.973053,30,-0.002877,0.041749,133,5,0.002757,0.005573,0.005635,0.005576,0.004474,0.004838,0.005192,0.004141,0.005156,0.005220,N,435.0,2,20814,16575,101997,16283,12132,54.0,6.9,10.0,0.0,0.0


In [73]:
osrm_df = pd.concat([osrm_test, orsm_train1, orsm_train2])[['id', 'total_distance', 'total_travel_time', 'number_of_steps']]
df = df.merge(osrm_df, on='id', how='left')

In [74]:
df['store_and_fwd_flag'] = 1 * (df.store_and_fwd_flag.values == 'Y')
df.head()

,direction,distance_dummy_manhattan,distance_haversine,dropoff_cluster,dropoff_datetime,dropoff_latitude,dropoff_longitude,dropoff_pca0,dropoff_pca1,holiday,holiday_after,holiday_prev,id,log_trip_duration,log_trip_duration_gby_dropoff_cluster,log_trip_duration_gby_pickup_cluster,log_trip_duration_gby_pickup_date,log_trip_duration_gby_pickup_hour,log_trip_duration_gby_pickup_week_hour,log_trip_duration_gby_vendor_id,log_trip_duration_gbys_pickup_cluster_dropoff_cluster,log_trip_duration_gbys_pickup_hour_dropoff_cluster,log_trip_duration_gbys_pickup_hour_pickup_cluster,passenger_count,pickup_cluster,pickup_date,pickup_datetime,pickup_hour,pickup_hour_weekofyear,pickup_latitude,pickup_longitude,pickup_minute,pickup_pca0,pickup_pca1,pickup_week_hour,pickup_weekday,speed,speed_gby_dropoff_cluster,speed_gby_pickup_cluster,speed_gby_pickup_date,speed_gby_pickup_hour,speed_gby_pickup_week_hour,speed_gby_vendor_id,speed_gbys_pickup_cluster_dropoff_cluster,speed_gbys_pickup_hour_dropoff_cluster,speed_gbys_pickup_hour_pickup_cluster,store_and_fwd_flag,trip_duration,vendor_id,total_trips_gby_pickup_cluster,total_trips_gby_dropoff_cluster,total_trips_gby_pickup_hour,total_trips_gby_pickup_week_hour,total_trips_gby_pickup_date,tempi,wspdi,visi,rain,snow,total_distance,total_travel_time,number_of_steps
0,99.970196,1.735433,1.498521,19,2016-03-14 17:32:30,40.765602,-73.964630,-0.009666,0.013695,0,0,0,id2875421,6.122493,6.381392,6.455874,6.445390,6.537261,6.469051,6.481609,6.160872,6.442969,6.554691,1,7,2016-03-14,2016-03-14 17:24:55,17,11,40.767937,-73.982155,24,0.007691,0.017053,17,0,0.003814,0.004180,0.004523,0.004966,0.004496,0.004700,0.005192,0.003821,0.003750,0.003837,0,455.0,2,56201,53443,109399,15664,11564,41.0,13.8,8.0,0.0,0.0,2009.1,164.9,5.0
1,-117.153768,2.430506,1.805507,28,2016-06-12 00:54:38,40.731152,-73.999481,0.027145,-0.018652,0,0,0,id2377394,6.498282,6.288792,6.362313,6.421458,6.422417,6.465794,6.450148,6.333634,6.177833,6.305238,1,10,2016-06-12,2016-06-12 00:43:35,0,23,40.738564,-73.980415,43,0.007677,-0.012371,144,6,0.003666,0.004712,0.004952,0.006094,0.006315,0.005577,0.005167,0.003321,0.005073,0.005878,0,663.0,1,35903,25396,75962,17358,10086,41.0,3.5,10.0,0.0,0.0,2513.2,332.0,6.0
2,-159.680165,8.203575,6.385098,88,2016-01-19 12:10:48,40.710087,-74.005333,0.034222,-0.039337,0,1,0,id3858529,7.661527,6.544065,6.534438,6.459165,6.520644,6.635141,6.481609,7.315407,6.664105,6.549867,1,62,2016-01-19,2016-01-19 11:35:24,11,3,40.763939,-73.979027,35,0.004803,0.012879,35,1,0.003862,0.005179,0.004360,0.004975,0.004470,0.003850,0.005192,0.005667,0.004512,0.004030,0,2124.0,2,56725,25421,97621,13937,11315,24.1,16.1,10.0,0.0,0.0,11060.8,767.6,16.0
3,-172.737700,1.661331,1.485498,30,2016-04-06 19:39:40,40.706718,-74.012268,0.041343,-0.042293,0,0,0,id3504673,6.063785,6.726676,6.529342,6.486666,6.441269,6.459194,6.481609,6.378557,6.676450,6.507031,1,5,2016-04-06,2016-04-06 19:32:31,19,14,40.719971,-74.010040,32,0.038342,-0.029194,67,2,0.003873,0.006205,0.005106,0.004863,0.004876,0.004696,0.005192,0.003796,0.005780,0.004844,0,429.0,2,29748,35647,128745,19519,12045,72.0,3.5,1.8,1.0,0.0,1779.4,235.8,4.0
4,179.473585,1.199457,1.188588,18,2016-03-26 13:38:10,40.782520,-73.972923,-0.002380,0.031071,0,0,0,id2181028,6.077642,6.225785,6.289475,6.398234,6.537807,6.444176,6.481609,5.437386,6.199833,6.290428,1,74,2016-03-26,2016-03-26 13:30:55,13,12,40.793209,-73.973053,30,-0.002877,0.041749,133,5,0.002757,0.005573,0.005635,0.005576,0.004474,0.004838,0.005192,0.004141,0.005156,0.005220,0,435.0,2,20814,16575,101997,16283,12132,54.0,6.9,10.0,0.0,0.0,1614.9,140.1,5.0


In [75]:
f_wo = ['id', 'log_trip_duration', 'pickup_date', 'pickup_datetime',
        'speed', 'trip_duration', 'dropoff_datetime']
f_to_use = [f for f in list(df) if f not in f_wo]

In [76]:
train = df[:train.shape[0]]
test = df[train.shape[0]:]

In [77]:
train.head()

,direction,distance_dummy_manhattan,distance_haversine,dropoff_cluster,dropoff_datetime,dropoff_latitude,dropoff_longitude,dropoff_pca0,dropoff_pca1,holiday,holiday_after,holiday_prev,id,log_trip_duration,log_trip_duration_gby_dropoff_cluster,log_trip_duration_gby_pickup_cluster,log_trip_duration_gby_pickup_date,log_trip_duration_gby_pickup_hour,log_trip_duration_gby_pickup_week_hour,log_trip_duration_gby_vendor_id,log_trip_duration_gbys_pickup_cluster_dropoff_cluster,log_trip_duration_gbys_pickup_hour_dropoff_cluster,log_trip_duration_gbys_pickup_hour_pickup_cluster,passenger_count,pickup_cluster,pickup_date,pickup_datetime,pickup_hour,pickup_hour_weekofyear,pickup_latitude,pickup_longitude,pickup_minute,pickup_pca0,pickup_pca1,pickup_week_hour,pickup_weekday,speed,speed_gby_dropoff_cluster,speed_gby_pickup_cluster,speed_gby_pickup_date,speed_gby_pickup_hour,speed_gby_pickup_week_hour,speed_gby_vendor_id,speed_gbys_pickup_cluster_dropoff_cluster,speed_gbys_pickup_hour_dropoff_cluster,speed_gbys_pickup_hour_pickup_cluster,store_and_fwd_flag,trip_duration,vendor_id,total_trips_gby_pickup_cluster,total_trips_gby_dropoff_cluster,total_trips_gby_pickup_hour,total_trips_gby_pickup_week_hour,total_trips_gby_pickup_date,tempi,wspdi,visi,rain,snow,total_distance,total_travel_time,number_of_steps
0,99.970196,1.735433,1.498521,19,2016-03-14 17:32:30,40.765602,-73.964630,-0.009666,0.013695,0,0,0,id2875421,6.122493,6.381392,6.455874,6.445390,6.537261,6.469051,6.481609,6.160872,6.442969,6.554691,1,7,2016-03-14,2016-03-14 17:24:55,17,11,40.767937,-73.982155,24,0.007691,0.017053,17,0,0.003814,0.004180,0.004523,0.004966,0.004496,0.004700,0.005192,0.003821,0.003750,0.003837,0,455.0,2,56201,53443,109399,15664,11564,41.0,13.8,8.0,0.0,0.0,2009.1,164.9,5.0
1,-117.153768,2.430506,1.805507,28,2016-06-12 00:54:38,40.731152,-73.999481,0.027145,-0.018652,0,0,0,id2377394,6.498282,6.288792,6.362313,6.421458,6.422417,6.465794,6.450148,6.333634,6.177833,6.305238,1,10,2016-06-12,2016-06-12 00:43:35,0,23,40.738564,-73.980415,43,0.007677,-0.012371,144,6,0.003666,0.004712,0.004952,0.006094,0.006315,0.005577,0.005167,0.003321,0.005073,0.005878,0,663.0,1,35903,25396,75962,17358,10086,41.0,3.5,10.0,0.0,0.0,2513.2,332.0,6.0
2,-159.680165,8.203575,6.385098,88,2016-01-19 12:10:48,40.710087,-74.005333,0.034222,-0.039337,0,1,0,id3858529,7.661527,6.544065,6.534438,6.459165,6.520644,6.635141,6.481609,7.315407,6.664105,6.549867,1,62,2016-01-19,2016-01-19 11:35:24,11,3,40.763939,-73.979027,35,0.004803,0.012879,35,1,0.003862,0.005179,0.004360,0.004975,0.004470,0.003850,0.005192,0.005667,0.004512,0.004030,0,2124.0,2,56725,25421,97621,13937,11315,24.1,16.1,10.0,0.0,0.0,11060.8,767.6,16.0
3,-172.737700,1.661331,1.485498,30,2016-04-06 19:39:40,40.706718,-74.012268,0.041343,-0.042293,0,0,0,id3504673,6.063785,6.726676,6.529342,6.486666,6.441269,6.459194,6.481609,6.378557,6.676450,6.507031,1,5,2016-04-06,2016-04-06 19:32:31,19,14,40.719971,-74.010040,32,0.038342,-0.029194,67,2,0.003873,0.006205,0.005106,0.004863,0.004876,0.004696,0.005192,0.003796,0.005780,0.004844,0,429.0,2,29748,35647,128745,19519,12045,72.0,3.5,1.8,1.0,0.0,1779.4,235.8,4.0
4,179.473585,1.199457,1.188588,18,2016-03-26 13:38:10,40.782520,-73.972923,-0.002380,0.031071,0,0,0,id2181028,6.077642,6.225785,6.289475,6.398234,6.537807,6.444176,6.481609,5.437386,6.199833,6.290428,1,74,2016-03-26,2016-03-26 13:30:55,13,12,40.793209,-73.973053,30,-0.002877,0.041749,133,5,0.002757,0.005573,0.005635,0.005576,0.004474,0.004838,0.005192,0.004141,0.005156,0.005220,0,435.0,2,20814,16575,101997,16283,12132,54.0,6.9,10.0,0.0,0.0,1614.9,140.1,5.0


In [78]:
train_2 = pd.read_pickle('input/process/train_df.pkl')
train_2.head()

,direction,distance_dummy_manhattan,distance_haversine,dropoff_cluster,dropoff_datetime,dropoff_latitude,dropoff_longitude,dropoff_pca0,dropoff_pca1,holiday,holiday_after,holiday_prev,id,log_trip_duration,log_trip_duration_gby_dropoff_cluster,log_trip_duration_gby_pickup_cluster,log_trip_duration_gby_pickup_date,log_trip_duration_gby_pickup_hour,log_trip_duration_gby_pickup_week_hour,log_trip_duration_gby_vendor_id,log_trip_duration_gbys_pickup_cluster_dropoff_cluster,log_trip_duration_gbys_pickup_hour_dropoff_cluster,log_trip_duration_gbys_pickup_hour_pickup_cluster,passenger_count,pickup_cluster,pickup_date,pickup_datetime,pickup_hour,pickup_hour_weekofyear,pickup_latitude,pickup_longitude,pickup_minute,pickup_pca0,pickup_pca1,pickup_week_hour,pickup_weekday,speed,speed_gby_dropoff_cluster,speed_gby_pickup_cluster,speed_gby_pickup_date,speed_gby_pickup_hour,speed_gby_pickup_week_hour,speed_gby_vendor_id,speed_gbys_pickup_cluster_dropoff_cluster,speed_gbys_pickup_hour_dropoff_cluster,speed_gbys_pickup_hour_pickup_cluster,store_and_fwd_flag,trip_duration,vendor_id,tempi,wspdi,visi,rain,snow,total_distance,total_travel_time,number_of_steps,total_trips_gby_pickup_cluster,total_trips_gby_dropoff_cluster,total_trips_gby_pickup_hour,total_trips_gby_pickup_week_hour,total_trips_gby_pickup_date
0,99.970196,1.735433,1.498521,95,2016-03-14 17:32:30,40.765602,-73.964630,-0.009995,0.013490,0,0,0,id2875421,6.122493,6.238885,6.429614,6.445390,6.537261,6.469051,6.481609,6.329711,6.234379,6.534852,1,11,2016-03-14,2016-03-14 17:24:55,17,11,40.767937,-73.982155,24,0.007345,0.016941,17,0,0.003814,0.004371,0.004710,0.004966,0.004496,0.004700,0.005192,0.003202,0.004238,0.004019,0,455.0,2,41.0,13.8,8.0,0.0,0.0,2009.1,164.9,5.0,50863.0,23606.0,109399,15664,11564
1,-117.153768,2.430506,1.805507,24,2016-06-12 00:54:38,40.731152,-73.999481,0.026989,-0.018660,0,0,0,id2377394,6.498282,6.318431,6.430564,6.421458,6.422417,6.465794,6.450148,6.699398,6.247273,6.350044,1,18,2016-06-12,2016-06-12 00:43:35,0,23,40.738564,-73.980415,43,0.007488,-0.012483,144,6,0.003666,0.004738,0.005003,0.006094,0.006315,0.005577,0.005167,0.003373,0.004948,0.006426,0,663.0,1,41.0,3.5,10.0,0.0,0.0,2513.2,332.0,6.0,12522.0,40461.0,75962,17358,10086
2,-159.680165,8.203575,6.385098,48,2016-01-19 12:10:48,40.710087,-74.005333,0.034177,-0.039307,0,1,0,id3858529,7.661527,6.650505,6.539139,6.459165,6.520644,6.635141,6.481609,7.294905,6.782596,6.585665,1,79,2016-01-19,2016-01-19 11:35:24,11,3,40.763939,-73.979027,35,0.004479,0.012751,35,1,0.003862,0.005451,0.004323,0.004975,0.004470,0.003850,0.005192,0.005831,0.004750,0.003939,0,2124.0,2,24.1,16.1,10.0,0.0,0.0,11060.8,767.6,16.0,63449.0,14597.0,97621,13937,11315
3,-172.737700,1.661331,1.485498,86,2016-04-06 19:39:40,40.706718,-74.012268,0.041313,-0.042225,0,0,0,id3504673,6.063785,6.716519,6.532783,6.486666,6.441269,6.459194,6.481609,6.112149,6.627157,6.479683,1,2,2016-04-06,2016-04-06 19:32:31,19,14,40.719971,-74.010040,32,0.038242,-0.029143,67,2,0.003873,0.006717,0.005320,0.004863,0.004876,0.004696,0.005192,0.004315,0.006269,0.005039,0,429.0,2,70.2,2.8,1.7,0.6,0.0,1779.4,235.8,4.0,29775.0,12871.0,128745,19519,12045
4,179.473585,1.199457,1.188588,45,2016-03-26 13:38:10,40.782520,-73.972923,-0.002801,0.030904,0,0,0,id2181028,6.077642,6.227446,6.308065,6.398234,6.537807,6.444176,6.481609,5.480282,6.230057,6.294643,1,30,2016-03-26,2016-03-26 13:30:55,13,12,40.793209,-73.973053,30,-0.003355,0.041579,133,5,0.002757,0.005528,0.005529,0.005576,0.004474,0.004838,0.005192,0.003979,0.005044,0.005200,0,435.0,2,54.0,6.9,10.0,0.0,0.0,1614.9,140.1,5.0,18626.0,29940.0,101997,16283,12132


In [79]:
print(train.shape)
print(train_2.shape)

(1458644, 62)
(1458644, 62)


In [80]:
y = train.log_trip_duration.values

In [81]:
train[f_to_use].head()

,direction,distance_dummy_manhattan,distance_haversine,dropoff_cluster,dropoff_latitude,dropoff_longitude,dropoff_pca0,dropoff_pca1,holiday,holiday_after,holiday_prev,log_trip_duration_gby_dropoff_cluster,log_trip_duration_gby_pickup_cluster,log_trip_duration_gby_pickup_date,log_trip_duration_gby_pickup_hour,log_trip_duration_gby_pickup_week_hour,log_trip_duration_gby_vendor_id,log_trip_duration_gbys_pickup_cluster_dropoff_cluster,log_trip_duration_gbys_pickup_hour_dropoff_cluster,log_trip_duration_gbys_pickup_hour_pickup_cluster,passenger_count,pickup_cluster,pickup_hour,pickup_hour_weekofyear,pickup_latitude,pickup_longitude,pickup_minute,pickup_pca0,pickup_pca1,pickup_week_hour,pickup_weekday,speed_gby_dropoff_cluster,speed_gby_pickup_cluster,speed_gby_pickup_date,speed_gby_pickup_hour,speed_gby_pickup_week_hour,speed_gby_vendor_id,speed_gbys_pickup_cluster_dropoff_cluster,speed_gbys_pickup_hour_dropoff_cluster,speed_gbys_pickup_hour_pickup_cluster,store_and_fwd_flag,vendor_id,total_trips_gby_pickup_cluster,total_trips_gby_dropoff_cluster,total_trips_gby_pickup_hour,total_trips_gby_pickup_week_hour,total_trips_gby_pickup_date,tempi,wspdi,visi,rain,snow,total_distance,total_travel_time,number_of_steps
0,99.970196,1.735433,1.498521,19,40.765602,-73.964630,-0.009666,0.013695,0,0,0,6.381392,6.455874,6.445390,6.537261,6.469051,6.481609,6.160872,6.442969,6.554691,1,7,17,11,40.767937,-73.982155,24,0.007691,0.017053,17,0,0.004180,0.004523,0.004966,0.004496,0.004700,0.005192,0.003821,0.003750,0.003837,0,2,56201,53443,109399,15664,11564,41.0,13.8,8.0,0.0,0.0,2009.1,164.9,5.0
1,-117.153768,2.430506,1.805507,28,40.731152,-73.999481,0.027145,-0.018652,0,0,0,6.288792,6.362313,6.421458,6.422417,6.465794,6.450148,6.333634,6.177833,6.305238,1,10,0,23,40.738564,-73.980415,43,0.007677,-0.012371,144,6,0.004712,0.004952,0.006094,0.006315,0.005577,0.005167,0.003321,0.005073,0.005878,0,1,35903,25396,75962,17358,10086,41.0,3.5,10.0,0.0,0.0,2513.2,332.0,6.0
2,-159.680165,8.203575,6.385098,88,40.710087,-74.005333,0.034222,-0.039337,0,1,0,6.544065,6.534438,6.459165,6.520644,6.635141,6.481609,7.315407,6.664105,6.549867,1,62,11,3,40.763939,-73.979027,35,0.004803,0.012879,35,1,0.005179,0.004360,0.004975,0.004470,0.003850,0.005192,0.005667,0.004512,0.004030,0,2,56725,25421,97621,13937,11315,24.1,16.1,10.0,0.0,0.0,11060.8,767.6,16.0
3,-172.737700,1.661331,1.485498,30,40.706718,-74.012268,0.041343,-0.042293,0,0,0,6.726676,6.529342,6.486666,6.441269,6.459194,6.481609,6.378557,6.676450,6.507031,1,5,19,14,40.719971,-74.010040,32,0.038342,-0.029194,67,2,0.006205,0.005106,0.004863,0.004876,0.004696,0.005192,0.003796,0.005780,0.004844,0,2,29748,35647,128745,19519,12045,72.0,3.5,1.8,1.0,0.0,1779.4,235.8,4.0
4,179.473585,1.199457,1.188588,18,40.782520,-73.972923,-0.002380,0.031071,0,0,0,6.225785,6.289475,6.398234,6.537807,6.444176,6.481609,5.437386,6.199833,6.290428,1,74,13,12,40.793209,-73.973053,30,-0.002877,0.041749,133,5,0.005573,0.005635,0.005576,0.004474,0.004838,0.005192,0.004141,0.005156,0.005220,0,2,20814,16575,101997,16283,12132,54.0,6.9,10.0,0.0,0.0,1614.9,140.1,5.0


In [82]:
Xtr, Xv, ytr, yv = train_test_split(train[f_to_use].values, y, test_size=0.2, random_state=1987)
dtrain = xgb.DMatrix(Xtr, label=ytr)
dvalid = xgb.DMatrix(Xv, label=yv)
dtest = xgb.DMatrix(test[f_to_use].values)
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

xgb_pars = {'min_child_weight': 50, 'eta': 0.3, 'colsample_bytree': 0.3, 'max_depth': 10,
            'subsample': 0.8, 'lambda': 1., 'nthread': -1, 'booster' : 'gbtree', 'silent': 1,
            'eval_metric': 'rmse', 'objective': 'reg:linear'}

In [83]:
# You could try to train with more epoch
model = xgb.train(xgb_pars, dtrain, 60, watchlist, early_stopping_rounds=50,
                  maximize=False, verbose_eval=10)

[0]	train-rmse:4.22586	valid-rmse:4.227
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[10]	train-rmse:0.400363	valid-rmse:0.413391
[20]	train-rmse:0.369842	valid-rmse:0.389191
[30]	train-rmse:0.362192	valid-rmse:0.387066
[40]	train-rmse:0.35703	valid-rmse:0.38578
[50]	train-rmse:0.352872	valid-rmse:0.384852


In [84]:
def rmsle(preds,y):
     return np.sqrt(np.mean(np.square(np.log1p(np.exp(preds))-np.log1p(np.exp(y)))))

In [85]:
# cv score
rmsle(model.predict(dvalid), yv)

0.38190723160688983

In [46]:
ytest = model.predict(dtest)
test.loc[:, 'trip_duration'] = np.exp(ytest) - 1
test[['id', 'trip_duration']].to_csv('output/ny_taxi_822.csv', index=False)